<a href="https://www.kaggle.com/code/sayoncamara/causal-analysis-of-rental-price-determinants?scriptVersionId=204506595" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

#  Causal Analysis of Rental Price Determinants: Policy Insights for Real Estate Interventions in Brussels

1. **Introduction**

In today's dynamic real estate market, understanding the factors that influence rental prices is critical for property owners, investors, and policymakers. Traditional models often provide predictions based on correlations, but they may not fully capture the causal relationships between different property features and rental prices. This project takes a step further by employing causal inference techniques to determine not only which features are most important in predicting rental prices but also how interventions like adding amenities or making structural improvements can impact rental returns.

The project begins with extensive exploratory data analysis (EDA), as the dataset used is notably dirty, with a significant portion of missing values. Careful imputation techniques were employed to handle these gaps and prepare the data for analysis. After cleaning the data, a monthly rental price prediction model was built to extract the most important predictive features using SHAP values, which helped highlight the top-ranking variables that contribute to rental price predictions.

However, while SHAP values reveal feature importance in a predictive sense, they do not provide insights into the causal effects of these features. To bridge this gap, we leverage CausalAnalysis from EconML to determine which features not only correlate with higher rents but also causally impact rental prices. By understanding the actual causal relationships, we can provide more reliable insights into how specific property features influence rental income.

In the next phase, we explore how property features—such as heating systems, solar panels, furnishing, and building conditions—affect rental prices through causal inference models. Using tools like policy trees and heterogeneity analysis, we dive deep into how these features impact different types of properties.

Ultimately, this project aims to offer actionable insights for real estate decision-makers, highlighting which property features have the greatest potential to increase rental prices and guiding cost-effective property improvements.

2. **Data exploration**

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder

# Set pandas option to display all columns
pd.set_option('display.max_columns', None)

# Load your dataset
data = pd.read_csv('/kaggle/input/monthly-rent-of-rented-flats-in-brussels/rental_df_dirty.csv')


In [ ]:
data

In [ ]:
# Calculate the percentage of missing values for each column
missing_percentage = data.isnull().mean() * 100

# Convert to a DataFrame for easier viewing, sorted by percentage
missing_percentage_data = missing_percentage.to_frame(name='Missing Percentage').sort_values(by='Missing Percentage', ascending=False)

# Display the result
print(missing_percentage_data)


- The dependent variable, Monthly rental price is in string form and has duplicates, other variables need to be cleaned as well. 
- The vast majority of variables have missing values even going up to 90%, I would lose a lot of information as I decided to simply remove them. Filling so many missing values with the mean would possiby distort the actual distribution of the variables  ,so i settled with the following solution, variables whose were missing more than 85% percent of that and were barely correlated to the monthly rental price were first removed. The rest of the variables that contained missing values were filled with an iterative imputer. An iterative imputer is a method used to fill in missing data by modeling each feature with missing values as a function of other features, and then iteratively refining these estimates until convergence. This approach captures relationships between features, often resulting in more accurate imputations compared to simpler methods like mean or median imputation.

3. **Feature engineering**

In [ ]:
import re


# The column to clean is 'Monthly rental price'

# Function to clean the price column
def clean_price(price_str):
    # Check if the value is a string, if not return it as is (or handle missing values)
    if isinstance(price_str, str):
        # Remove euro symbol and any spaces
        price_str = re.sub(r'[€\s]', '', price_str)
        
        # Extract the first number (assumes format like '750750' or '12001200')
        match = re.match(r'(\d{1,3}(?:,\d{3})?)', price_str)
        
        if match:
            price_str = match.group(0)
            # Convert comma to empty string for thousand separator
            price_str = price_str.replace(',', '')
            # Convert to numeric
            return pd.to_numeric(price_str)
        else:
            return None  # Return None if no match found
    else:
        return price_str  # If it's not a string, return the value as is (e.g. NaN)

# Apply the cleaning function ONLY to the 'Monthly rental price' column
data['Monthly rental price'] = data['Monthly rental price'].apply(clean_price)

# Display the DataFrame with the cleaned 'Monthly rental price' column


In [ ]:
columns_to_remove = [
    "Professional space",
    "Living room",
    "Basement",
    "Primary energy consumption",
    "Reference number of the EPC report",
    "CO₂ emission",
    "Yearly theoretical total energy consumption",
    "Monthly costs",
    
    "As built plan",
    "Address",
    "Website",
    "External reference",
    "agency",
    "living room",
    "Dining room",
    "Laundry room",
    "Caretaker",
    "Secure access / alarm",
    "internet",
    "Conformity certification for fuel tanks",
    "Possible priority purchase right",
    "Street frontage width",
    "description",
    "Basement surface",
    "Neighbourhood or locality",
    "How many fireplaces?",
    "Terrace",
    "Garden surface",
    "Garden orientation",
    "Planning permission obtained",
    "Subdivision permit",
    "Flood zone type",
    "Surroundings type",
    "Virtual visit",
    "Property name",
    "Proceedings for breach of planning regulations",
    "Professional space surface",
    "Obligation to build",
    "Attic surface",
    "Office surface",
    "Attic",
    "Surface of the plot",
    "Connection to sewer network",
    "Gas, water & electricity",
    "Total ground floor buildable",
    "Latest land use designation",
    "Garden",
    "price",
    
    "Tenement building",
    "E-level (overall energy performance)",
    "Agent's name",
    "Land is facing street",
    "Wooded land",
    "Plot at rear",
    "Flat land",
    "Bedroom 4 surface",
    "EPC description",
    "Terrace orientation",
    "Bedroom 5 surface",
    "Width of the lot on the street",
    "Isolated",
    "Extra information",
    "Available date",
    "Number of annexes"
]

# Remove the columns
df = data.drop(columns=columns_to_remove, errors='ignore')

In [ ]:
# 1. Cleaning the 'space' column
def clean_space(space_entry):
    if pd.isnull(space_entry):
        return np.nan
    # Extract the square meters using regex
    match = re.search(r'(\d+)\s*m²', space_entry)
    if match:
        return int(match.group(1))  # Extract the numeric value
    return np.nan

# Apply the cleaning function to the 'space' column
df['total_space'] = df['space'].apply(clean_space)

# 2. Cleaning the 'address' column
def clean_address(address_entry):
    if pd.isnull(address_entry) or "Ask for the exact address" in address_entry:
        return np.nan  # Treat these as missing values
    # Extract postal codes (first four digits from the address)
    match = re.search(r'\b\d{4}\b', address_entry)
    if match:
        return int(match.group(0))  # Extract the postal code
    return np.nan

# Apply the cleaning function to the 'address' column
df['zip_code'] = df['address'].apply(clean_address)

# Now, you can drop the original columns if necessary
df.drop(columns=['space', 'address'], inplace=True)

df

I've written a custom cleaning process to extract usable data from the 'space' and 'address' columns by identifying square meter values and postal codes, respectively. The cleaned data is stored in new columns, and the original columns are removed once the transformation is complete.








In [ ]:

# Function to extract numeric value from the string
def extract_numeric(value):
    if pd.isna(value) or value == 'MISSING':
        return value
    return pd.to_numeric(value.split()[0], errors='coerce')

# Apply the function to the 'Living area' column
df['Living area'] = df['Living area'].apply(extract_numeric)

# Convert 'Living area' column to numeric, keeping 'MISSING' as is
df['Living area'] = pd.to_numeric(df['Living area'], errors='coerce').fillna(df['Living area'])
df = df.drop('Living area', axis=1)

The dataset underwent significant cleaning to address inconsistencies and prepare it for modeling. 
First, numeric values were extracted from mixed text entries, such as in the 'Living area' column, while preserving missing data. Binary categorical columns like 'Furnished' and 'Elevator' were converted from 'Yes/No' to 1/0 format. Surface area columns, including 'Terrace surface' and 'Bedroom 1 surface,' were cleaned by extracting only numeric values, leaving missing values unchanged. Finally, key categorical variables like 'Kitchen type' and 'Building condition' were label-encoded, while ensuring NaN values were retained.

In [ ]:
# List of binary categorical columns
binary_columns = [
    'Dressing room', 'Office', 'Small pet-friendly', 'Armored door', 'Heat pump',
    'Photovoltaic solar panels', 'Thermic solar panels', 'Common water heater', 
    'Double glazing', 'Furnished', 'Intercom', 'Visio phone', 'Elevator', 
    'Accessible for disabled people', 'Air conditioning', 'TV cable', 'Jacuzzi',
    'Sauna', 'Swimming pool', 'Internet'
]

# Assuming your DataFrame is called 'df'
# Convert 'Yes'/'No' to 1/0 using a map function
for col in binary_columns:
    df[col] = df[col].map({'Yes': 1, 'No': 0})




In [ ]:

# List of columns to clean
columns_to_clean = ['Living room surface', 'Bedroom 1 surface', 'Bedroom 2 surface', 
                    'Terrace surface', 'Kitchen surface', 'Bedroom 3 surface']

# Function to extract numeric part from the string
def extract_numeric(value):
    if pd.isna(value):
        return value  # Keep NaN as is
    # Use regex to find any sequence of digits with optional decimal points
    match = re.search(r'[\d]+(?:\.\d+)?', str(value))
    if match:
        # Convert the extracted number to a float
        return float(match.group(0))
    else:
        return pd.NA  # Return NaN if no numeric value found

# Apply the function to the specified columns
df[columns_to_clean] = df[columns_to_clean].applymap(extract_numeric)



In [ ]:
# List of columns to label encode
columns_to_encode = ['Kitchen type', 'Energy class', 'Heating type', 
                     'Type of building', 'listing_type', 'Available as of','Building condition']

# Initialize the LabelEncoder
le = LabelEncoder()

# Apply LabelEncoder and preserve NaN values
for column in columns_to_encode:
    if column in df.columns:
        # Save the NaN mask to remember which values were NaN
        nan_mask = df[column].isna()
        
        # Convert non-NaN values to strings and apply LabelEncoder
        df[column] = df[column].astype(str)
        df[column] = le.fit_transform(df[column])
        
        # Restore the NaN values after encoding
        df.loc[nan_mask, column] = pd.NA



In [ ]:

# Step 1: Convert non-NaN values to integers and then to strings
df['zip_code'] = df['zip_code'].apply(lambda x: str(int(x)) if pd.notnull(x) else x)

# Step 2: Ensure the column is of type 'object' (string)
df['zip_code'] = df['zip_code'].astype('object')

The data was preprocessed using a combination of ordinal encoding and iterative imputation to handle missing values in both numeric and categorical columns. First, numeric and categorical columns were separated. Categorical variables were encoded using an OrdinalEncoder, transforming them into numeric format. For missing values in numeric columns, an IterativeImputer was used with a LGBMRegressor as the estimator, while for categorical columns, a LGBMClassifier imputed missing values. After imputation, the results were reassembled into DataFrames, and the categorical columns were inverse-transformed back to their original labels. Finally, the numeric and categorical columns were recombined to form the final dataset, with all missing values imputed and ready for modeling.

In [ ]:

from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OrdinalEncoder



# Step 1: Separate numeric and categorical columns
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
categorical_columns = df.select_dtypes(exclude=['number']).columns.tolist()

# Step 2: Ordinal encode categorical columns (convert strings to integers)
encoder = OrdinalEncoder()
df[categorical_columns] = encoder.fit_transform(df[categorical_columns])

# Step 3: Imputation for numeric columns using LGBMRegressor
numeric_imputer = IterativeImputer(estimator=LGBMRegressor(n_estimators=100, random_state=0,verbose=-1 ),
                                   max_iter=10, random_state=0)

# Step 4: Imputation for categorical columns using LGBMClassifier
categorical_imputer = IterativeImputer(estimator=LGBMClassifier(n_estimators=100, random_state=0,verbose=-1 ),
                                       max_iter=10, random_state=0)

# Step 5: Fit and transform each type of column separately
# Impute numeric columns
imputed_numeric_array = numeric_imputer.fit_transform(df[numeric_columns])

# Impute categorical columns
imputed_categorical_array = categorical_imputer.fit_transform(df[categorical_columns])

# Step 6: Convert the results back to DataFrames
imputed_numeric_df = pd.DataFrame(imputed_numeric_array, columns=numeric_columns)
imputed_categorical_df = pd.DataFrame(imputed_categorical_array, columns=categorical_columns)

# Step 7: Inverse transform categorical columns back to original categories
imputed_categorical_df[categorical_columns] = encoder.inverse_transform(imputed_categorical_df)

# Step 8: Combine the numeric and categorical columns back into a single DataFrame
final_imputed_df = pd.concat([imputed_numeric_df, imputed_categorical_df], axis=1)




In [ ]:
# Define the binary columns based on your provided list
binary_columns = [
    'Office', 'Small pet-friendly', 'Armored door', 'Heat pump','Thermic solar panels', 'Photovoltaic solar panels','Dressing room','Elevator',
    'Thermic solar Panels', 'Common water heater', 'Double glazing', 'Furnished', 'Intercom',
    'Visio phone', 'Accessible for disabled people', 'Air conditioning', 'TV cable', 
    'Jacuzzi', 'Sauna', 'Swimming pool', 'Internet'
]

# Step 1: Apply thresholding for binary columns in the DataFrame `final_imputed_df`
for col in binary_columns:
    if col in final_imputed_df.columns:  # Ensure the column exists in the DataFrame
        final_imputed_df[col] = np.where(final_imputed_df[col] < 0.5, 0, 1)




In [ ]:
# Define the multi-class numeric columns (discrete variables) that were imputed
multi_class_columns = [
    'Kitchen type', 'Bedrooms', 'Bathrooms', 'Toilets', 'Heating type', 
    'Type of building', 'Number of floors', 'Number of frontages', 
    'Outdoor parking spaces', 'Construction year', 'Building condition', 
    'Available as of', 'Shower rooms', 'Covered parking spaces'
]

# Step 1: Apply rounding for all the multi-class numeric columns
for col in multi_class_columns:
    if col in final_imputed_df.columns:  # Ensure the column exists in the DataFrame
        # Round to nearest integer and convert to integer type
        final_imputed_df[col] = np.round(final_imputed_df[col]).astype(int)



In [ ]:
# List of columns to one-hot encode
columns_to_encode = [
    'Office', 'Small pet-friendly', 'Armored door', 'Heat pump', 'Photovoltaic solar panels',
    'Thermic solar panels', 'Common water heater', 'Double glazing', 'Furnished', 'Intercom',
    'Visio phone', 'Accessible for disabled people', 'Air conditioning', 'TV cable', 
    'Jacuzzi', 'Sauna', 'Swimming pool', 'Internet', 'zip_code'
]

# Step 1: Apply one-hot encoding to the specified columns
final_encoded_df = pd.get_dummies(final_imputed_df, columns=columns_to_encode, drop_first=True)




This code performs two key data preprocessing steps: first, it transforms a set of specified columns containing float values into strict binary features (0 or 1) using a 0.5 threshold, ensuring features like 'Office', 'Swimming pool', and other amenities are represented as clear binary indicators; then, it applies one-hot encoding to these binary features along with the 'cleaned_address' categorical column , resulting in a transformed dataset where all categorical variables are properly encoded for machine learning model consumption

4. **Predictive modeling**

This code implements a machine learning pipeline for predicting monthly rental prices: it splits the dataset into features (X) and target variable (y), creates training and test sets, then uses LightGBM (a gradient boosting framework) with GridSearchCV to automatically find the best combination of hyperparameters (learning rate and max depth) through 5-fold cross-validation, ultimately training a model and evaluating its performance on the test set.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMRegressor



# Step 1: Separate the features (X) and the target (y)
X = final_encoded_df.drop(columns=['Monthly rental price'])
y = final_encoded_df['Monthly rental price']

# Step 2: Split data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Step 3: Train a LightGBM regression model with GridSearchCV for hyperparameter tuning
estimator = LGBMRegressor(random_state=0,verbose=-1)#




# Define a parameter grid to search over
param_grid = {
    "learning_rate": [0.1, 0.05, 0.01],
    "max_depth": [3, 5, 10]
}

# Use GridSearchCV to search for the best parameters
search = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)  # 5-fold cross-validation
search.fit(x_train, y_train)

# Step 4: Display the best parameters and evaluate model performance
print("Best estimator: ", search.best_params_)
print("Test set score: ", search.best_estimator_.score(x_test, y_test))

This code implements a machine learning pipeline using LightGBM to predict monthly rent prices, followed by an explanation of the model's predictions using SHAP (SHapley Additive exPlanations) values. The process involves training a LightGBM model with optimized parameters found through GridSearchCV, running for 100 boosting rounds on the training data. Then, a SHAP TreeExplainer is created to analyze how each feature contributes to the model's predictions, generating SHAP values for the test dataset. The resulting summary plot displays these feature impacts, where each point represents a sample, colored by the feature value (blue for low, red for high), and positioned on the x-axis according to its SHAP value (impact on prediction). The visualization reveals that 'total_space' has the strongest influence on rent predictions, followed by features like 'Toilets' and 'Terrace surface', while features such as 'Sauna_1' and 'Bedroom 3 surface' show minimal impact on the predicted rent prices.

In [ ]:
import shap
import lightgbm as lgb



# Convert the data to LightGBM Dataset format
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test, reference=lgb_train)

# Get the best parameters from GridSearchCV
best_params = search.best_params_

# Create and train a new model with the best parameters
model = lgb.train(best_params, lgb_train, num_boost_round=100)

# Now create the SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values
shap_values = explainer.shap_values(x_test)

# Visualize the results
shap.summary_plot(shap_values, x_test)

In [ ]:
pip install econml

5. **Causal modeling**

In [ ]:
from econml.solutions.causal_analysis import CausalAnalysis

# List of categorical features (add any other categorical features from your dataset)
categorical = [
    'Furnished_1', 'listing_type', 'Heating type', 'Type of building', 
    'Armored door_1', 'Elevator', 'Small pet-friendly_1', 'Office_1','Sauna_1', 
    'Accessible for disabled people_1', 'Swimming pool_1', 'Internet_1','Heat pump_1', 'Kitchen type','Double glazing_1','zip_code_1030', 'zip_code_1040', 'zip_code_1050', 'zip_code_1060', 'zip_code_1070', 'zip_code_1080', 'zip_code_1081', 'zip_code_1082', 'zip_code_1083', 'zip_code_1090', 'zip_code_1120', 'zip_code_1140', 'zip_code_1150', 'zip_code_1160', 'zip_code_1170', 'zip_code_1180', 'zip_code_1190', 'zip_code_1200', 'zip_code_1210', 'zip_code_1700', 'zip_code_2800', 'zip_code_8500', 'zip_code_9300', 'zip_code_9660', 'zip_code_9700','Outdoor parking spaces','Common water heater_1','Thermic solar panels_1','Photovoltaic solar panels_1', 'Air conditioning_1'
    # Add more as needed
]

# Heterogeneity features (features where treatment effects might vary across subgroups)
hetero_cols = ['Energy class', 'Building condition', 'Construction year']

# Initialize causal analysis
ca = CausalAnalysis(
    feature_inds=['Floor', 'Bedrooms', 'Bathrooms', 'Toilets','Number of frontages', 
                  'Terrace surface',  
                  'total_space','Outdoor parking spaces', ]+categorical ,
    categorical=categorical,
    heterogeneity_inds=hetero_cols,
    classification=False,
    nuisance_models="automl",
    heterogeneity_model="linear",
    n_jobs=-1,
    random_state=123,
    upper_bound_on_cat_expansion=6
)

# Fit the model
# Fit the causal analysis model
ca.fit(x_train, y_train)

In essence the code is ranking features by their causal importance to the outcome, based on the p-value of the causal effect. Features with smaller p-values are more likely to have a significant impact on the target variable, providing valuable insights into which factors are the most influential in your causal analysis. We can already spot a difference in feature importance in the causal model in comparison to the predictive model. Zip codes tend to have bigger causal impact on prices than would be assumed from the predictive model for example.

In [ ]:
# get global causal effect ordered by causal importance (pvalue)
global_summ = ca.global_causal_effect(alpha=0.05)
global_summ.sort_values(by="p_value")

This helper function effectively displays the causal effect size and significance of each feature. Features with significant effects (indicated by asterisks) stand out, making this an informative visualization for understanding which features have a statistically meaningful impact on the outcome.

In [ ]:
# helper function to plot error bar
def errorbar(res):
    xticks = res.index.get_level_values(0)
    lowererr = res["point"] - res["ci_lower"]
    uppererr = res["ci_upper"] - res["point"]
    xticks = [
        "{}***".format(t)
        if p < 1e-6
        else ("{}**".format(t) if p < 1e-3 else ("{}*".format(t) if p < 1e-2 else t))
        for t, p in zip(xticks, res["p_value"])
    ]
    plot_title = "Direct Causal Effect of Each Feature with 95% Confidence Interval, "
    plt.figure(figsize=(15, 5))
    plt.errorbar(
        np.arange(len(xticks)),
        res["point"],
        yerr=[lowererr, uppererr],
        fmt="o",
        capsize=5,
        capthick=1,
        barsabove=True,
    )
    plt.xticks(np.arange(len(xticks)), xticks, rotation=45)
    plt.title(plot_title)
    plt.axhline(0, color="r", linestyle="--", alpha=0.5)
    plt.ylabel("Average Treatment Effect")

In [ ]:
errorbar(global_summ)

 In the following section, we are going to use the presence of a thermic solar pan as an example to learn how different type of appartments  may increase the monthly rental price with the addition of a solar panel by looking a decision tree. 
 
 "Conditional average treatment effect ( CATE ) is a crucial concept in causal inference. It focuses on estimating treatment effects for specific subgroups or individuals, allowing for more personalized interventions by accounting for heterogeneity across different subpopulations."

The strongest treatment effects (highest CATE means) appear in newer properties (post-1955) with better conditions
Properties in poor condition show more varied effects (higher standard deviations)
The most recent properties (built after 1992) show the highest positive treatment effect (41.922)

This suggests that the causal impact  varies significantly based on the property's age and condition, with newer properties generally showing stronger positive effects on rental prices.

In [ ]:
plt.figure(figsize=(12, 8))
ca.plot_heterogeneity_tree(
    x_test,
    "Thermic solar panels_1",
    max_depth=2,
    min_impurity_decrease=1e-6,
    min_samples_leaf = 5
)